In [36]:
# concept : Align EEG, text and image spaces

'''
eeg encoder(저장된 MAE encoder를 사용) + project layer -> 차원 맞춘다음에, eeg embedding 만들기 -> 해당 임베딩과

imagenet을 통해서 paired image를 clip image encoder에 넣은다음에 만들어진 임베딩과

cosine similarity 계산을 통한 학습 -> loss를 torch.nn.CosineEmbeddingLoss ? 혹은 그냥 torch안에 있는 torch.nn.functional.cosine_similarity

torch.nn.CosineEmbeddingLoss(margin=0.0, size_average=None, reduce=None, reduction='mean')

'''
# 이 셀에서 argparse 대신 직접 매개변수를 설정하여 실행할 수 있습니다.
from dataset import eeg_pretrain_dataset  # dataset.py 파일에서 eeg_pretrain_dataset 클래스 import

import torch
import numpy as np
import copy
from torch.utils.data import DataLoader
import torch.nn as nn
import torch.nn.functional as F
from sc_mbm.mae_for_eeg import MAEforEEG
# from dc_ldm.modules.encoders import FrozenCLIPTextEmbedder

def fmri_transform(x, sparse_rate=0.2):
    # x: 1, num_voxels
    x_aug = copy.deepcopy(x)
    idx = np.random.choice(x.shape[0], int(x.shape[0]*sparse_rate), replace=False)
    x_aug[idx] = 0
    return torch.FloatTensor(x_aug)

# 매개변수 설정
path = 'data/processed/eegData_npy'
roi = 'VC'
patch_size = 4
transform = fmri_transform
aug_times = 1  # aug_times를 1로 설정하여 데이터를 한 번만 가져오도록 합니다.
num_sub_limit = None  # num_sub_limit를 None으로 설정하여 제한 없이 모든 데이터를 사용합니다.
include_hcp = True
batch_size = 4


# 데이터셋 초기화
dataset_pretrain = eeg_pretrain_dataset(path=path, roi=roi, patch_size=patch_size,
                                        transform=transform, aug_times=aug_times, 
                                        num_sub_limit=num_sub_limit, include_hcp=include_hcp)

dataloader_eeg = DataLoader(dataset_pretrain, batch_size=batch_size, shuffle=False)

# 데이터셋 크기와 데이터 길이 출력
print(f'Dataset size: {len(dataset_pretrain)}')
print(f'Time len: {dataset_pretrain.data_len}')
print(dataset_pretrain[0]['eeg'].shape)

Dataset size: 11965
Time len: 512
torch.Size([128, 512])


In [37]:
mae_model = MAEforEEG()

checkpoint = torch.load('pretrains/eeg_pretrain/checkpoint.pth')
mae_model.load_state_dict(checkpoint, strict=False)
mae_model.eval()

class MAEwithProjection(nn.Module):
    def __init__(self, mae_model):
        super().__init__()
        self.mae_model = mae_model  # 기존의 MAEforEEG 모델을 가져옵니다.
        self.projection_layer = nn.Linear(1024, 512)  # Projection layer를 정의합니다.

    def forward(self, imgs, mask_ratio=0.75):
        latent, _, _ = self.mae_model.forward_encoder(imgs, mask_ratio)
        embedding = self.projection_layer(latent[:, 0, :])
        return embedding

mae_with_projection = MAEwithProjection(mae_model)

for batch in dataloader_eeg:
    eeg_data = batch['eeg']
    break 

# Projection을 통해 EEG 데이터의 임베딩을 계산합니다.
with torch.no_grad():
    eeg_embedding = mae_with_projection(eeg_data)

print(eeg_embedding.shape)

torch.Size([4, 512])


In [38]:
checkpoint = torch.load('pretrains/eeg_pretrain/checkpoint.pth')
mae_model.load_state_dict(checkpoint, strict=False)
mae_model

MAEforEEG(
  (patch_embed): PatchEmbed1D(
    (proj): Conv1d(128, 1024, kernel_size=(4,), stride=(4,))
  )
  (blocks): ModuleList(
    (0): Block(
      (norm1): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
      (attn): Attention(
        (qkv): Linear(in_features=1024, out_features=3072, bias=True)
        (attn_drop): Dropout(p=0.0, inplace=False)
        (proj): Linear(in_features=1024, out_features=1024, bias=True)
        (proj_drop): Dropout(p=0.0, inplace=False)
      )
      (drop_path): Identity()
      (norm2): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
      (mlp): Mlp(
        (fc1): Linear(in_features=1024, out_features=4096, bias=True)
        (act): GELU(approximate=none)
        (drop1): Dropout(p=0.0, inplace=False)
        (fc2): Linear(in_features=4096, out_features=1024, bias=True)
        (drop2): Dropout(p=0.0, inplace=False)
      )
    )
    (1): Block(
      (norm1): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
      (attn): Att